## 1. Collect CodeSearchNet Repositories

In [35]:
import json

import pandas as pd
from pathlib import Path
pd.set_option('max_colwidth',300)
from pprint import pprint

In [41]:
!wget https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
!mkdir CodeSearchNet
!unzip python.zip -d CodeSearchNet

--2021-12-01 16:33:31--  https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.141.54
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.141.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 940909997 (897M) [application/zip]
Saving to: ‘python.zip.2’

python.zip.2        100%[===================>] 897.32M  44.0MB/s    in 17s     

2021-12-01 16:33:48 (54.2 MB/s) - ‘python.zip.2’ saved [940909997/940909997]

mkdir: cannot create directory ‘CodeSearchNet’: File exists
Archive:  python.zip
replace CodeSearchNet/python/final/jsonl/train/python_train_9.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace CodeSearchNet/python/final/jsonl/train/python_train_12.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [42]:
python_files = sorted(Path('CodeSearchNet/python').glob('**/*.gz'))

In [43]:
print(python_files)

[PosixPath('CodeSearchNet/python/final/jsonl/test/python_test_0.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_0.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_1.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_10.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_11.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_12.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_13.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_2.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_3.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_4.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_5.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_6.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_7.jsonl.gz

In [44]:
columns_long_list = ['repo', 'path', 'url', 'code', 
                     'code_tokens', 'docstring', 'docstring_tokens', 
                     'language', 'partition']

def jsonl_list_to_dataframe(file_list, columns=columns_long_list):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat([pd.read_json(f, 
                                   orient='records', 
                                   compression='gzip',
                                   lines=True)[columns] 
                      for f in file_list], sort=False)

In [45]:
columns_repo = ['repo']

pydf = jsonl_list_to_dataframe(python_files, columns=columns_repo)

In [46]:
pydf = pydf.drop_duplicates().reset_index(drop=True)

In [ ]:
print(pydf.shape)
pydf.head(13590)

In [48]:
pydf.to_pickle("repos.pkl")

## 2. Collect diff and commits

In [49]:
!mkdir repos
!pip install pydriller
!pip install pandas
!pip install spacy

mkdir: cannot create directory ‘repos’: File exists


In [50]:
from pydriller import *
import pandas as pd
import nltk
import spacy
import re
from functools import reduce
# spacy.cli.download("en_core_web_sm")

In [51]:
repodf = pd.read_pickle("repos.pkl")
print(repodf.shape)
spacy_tokenizer = spacy.load("en_core_web_sm")
diff_tokenizer = nltk.tokenize.WordPunctTokenizer()

(13590, 1)


In [93]:
import re
def basic_filter(message):
    return message.split("\n", 1)[0].strip()

# Remove [label] in front of commit if exists
def label_filter(message):
    if (message.startswith('[')):
        end_bracket_index = message.find(']')
        if (end_bracket_index == -1):
            return None
        return message[:end_bracket_index+1]
    return message

def camel_case_split(str):
    return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)


def case_splitter(token):
    return list(map(lambda x: x.lower(), camel_case_split(token[0].upper() + token[1:])))
    

def parse_repo_commits(repo_name, commit_limit=50):
    data = []
    commit_count = 0
    for commit in Repository(
        f"https://github.com/{repo_name}",
        only_modifications_with_file_types=[".py"],
        only_no_merge=True,
        order='reverse'
    ).traverse_commits():
        if (commit_count >= commit_limit): break
        line = basic_filter(commit.msg)
        line = label_filter(line)
        
        if (line is None):
            print(f"[DEBUG] Label filter return None for repo {repo} and hash {commit.hash}")
        
        line = line.replace('_', ' ')
        
        # Only alphabet and blank characters
        if (not line.isascii() or not all([c.isalpha() or c.isspace() for c in line])):
            continue
        
        '''
        if (not line.isascii()):                     # Ignore non-English
            continue
        
        if ('@' in line and not 'decorat' in line):  # Ignore Github mentions
            continue
        
        if ('#' in line):                            # Ignore Github issue
            continue
        '''
        
        tokens = spacy_tokenizer(line)
        
        # VERB filter
        if (tokens[0].pos_ != 'VERB'):
            continue
        
        tokens = reduce(lambda a,b: a+b, map(case_splitter, [token.text for token in tokens]), [])
        
        if (len(tokens) < 3 or len(tokens) > 30):
            continue
        
        # Check if changed files are python
        file_failed = False
        
        for mf in commit.modified_files:
            if (not mf.filename.endswith(".py")):
                file_failed = True
                break
        
        if (file_failed):
            continue
        def diff_processing(mf):
            print(mf.diff)
            diff = '\n'.join(map(lambda x: x[1], filter(lambda y: y[0] % 2 == 0, enumerate(mf.diff.split("@@")))))
            diff = diff.replace('\n+', '\n<add>').replace('\n-', '\n<del>')
            #TODO
            #replace_symbol_in_string = ex)url
            replace_number = re.compile(r"""
            (?:
              0x[0-9A-Fa-f]+        #hexadecimal number
              |[0-9]+               #decimal number
            )
            #(?:\.[0-9]+)?           #fraction(optional)
            """,re.VERBOSE)
            diff = replace_number.sub("<number>",diff)
            diff = re.sub(r"(?:\n[ \t\r\f\v]*)+","\n",diff) #Join continuous row change
            print(diff)
            token_regex = r"""(?x)
             <(?:add|del|number)>   #Filtered eariler
            #|[-+*/^&~|=%!]=?        #Symbols which can join with equal
            #|[<>]{1,2}              #neq and bit shift symbols
            #|#+                     #Comment symbol
            #|[@?$]                  #Other symbols
            |(?:[#][\s]*)+           #Quotation
            |(?:[-+*/^&~|=%!<>@?$][\s]*)+     #Sequence of symbols
            |[\n]                    #Change row
            |[a-zA-Z]+               #General text
            """
            #'"`,.;:()[]{}_ not included
            token_initial = nltk.tokenize.regexp_tokenize(diff,token_regex)
            token_camel_case_split = reduce(lambda a,b:a+b,map(lambda a: case_splitter(a) if 97<=ord(a[0].lower())<122 else [a], token_initial),[])
            print(token_camel_case_split)
            input()
            return diff
        
        print(tokens)
        diff = ''.join(map(diff_processing, commit.modified_files))
        
        data.append([repo_name, commit.hash, ' '.join(tokens), diff])
        commit_count += 1

    return pd.DataFrame(data, columns=["repo", "hash", "commit_messsage", "diff"])

In [94]:
import time
start = time.time()
df = parse_repo_commits("soimort/you-get")
end = time.time()
print(end - start)
df.head(3)

['add', 'hdr', 'support', 'for', 'bilibili']
@@ -12,6 +12,8 @@ class Bilibili(VideoExtractor):
 
     # Bilibili media encoding options, in descending quality order.
     stream_types = [
+        {'id': 'hdflv2', 'quality': 125, 'audio_quality': 30280,
+         'container': 'FLV', 'video_resolution': '3840p', 'desc': '真彩 HDR'},
         {'id': 'hdflv2_4k', 'quality': 120, 'audio_quality': 30280,
          'container': 'FLV', 'video_resolution': '2160p', 'desc': '超清 4K'},
         {'id': 'flv_p60', 'quality': 116, 'audio_quality': 30280,


class Bilibili(VideoExtractor):
# Bilibili media encoding options, in descending quality order.
stream_types = [
<add>        {'id': 'hdflv<number>', 'quality': <number>, 'audio_quality': <number>,
<add>         'container': 'FLV', 'video_resolution': '<number>p', 'desc': '真彩 HDR'},
{'id': 'hdflv<number>_<number>k', 'quality': <number>, 'audio_quality': <number>,
'container': 'FLV', 'video_resolution': '<number>p', 'desc': '超清 <number>K'},
{'id': 'f

KeyboardInterrupt: ignored

In [ ]:
import multiprocessing

data = []

def f(repo):
    df = parse_repo_commits(repo)
    df.to_pickle(f"./repos/{repo.replace('/', '+')}.pkl")
    print(repo, "Done")
    return df

pool = multiprocessing.Pool()
outputs = pool.map(f, repodf['repo'][:3])
pd.concat(outputs).to_pickle("data.pkl")

In [ ]:
df = pd.read_pickle("data.pkl")
df.head(3)